In [ ]:
import datetime
import sys

import numpy
import scipy.stats
%matplotlib widget
from matplotlib import pyplot
from matplotlib.dates import num2date, date2num
import netCDF4
from ipywidgets import widgets

sys.path.append('../extern/fabm-mizer/python')
import mizer

## Starting from a constant plankton size spectrum

In [ ]:
def run(nday=36500, c_ini=1., verbose=True, **parameter_overrides):
    parameters = dict(
        w_min=1e-3,
        w_inf=1e6,
        nclass=100,
        T_dependence=1,
        T_ref=13.,
        E_a=0.63,
        beta=100,
        sigma=float(numpy.log(10.)),
        gamma=156,
        q=0.82,
        #n=0.82, #0.93,
        alpha=0.2,
        z0_type=1,
        z0pre=0.1,
        z0exp=-0.25,
        w_s=1000.,
        z_s=0.3,
        z_spre=0.2,
        ks=0.,
        SRR=0,
        recruitment=0.,
        h=1e9, #30
        fishing_type=1,
        w_minF=1.25, # Blanchard et al 2012
        F=0.4 #4 #8  # note: need to put double the intended value due to fisheries equation!
    )
    parameters.update(parameter_overrides)

    prey_collection = mizer.PreyCollection(mizer.Prey('plankton', [1e-14, 1e-3], c_ini))
    prey_collection = mizer.GriddedPreyCollection(prey_collection, extend=False)

    # create mizer model
    m = mizer.Mizer(prey=prey_collection, parameters=parameters, temperature=13., depth=50., recruitment_from_prey=2, verbose=False)

    # time integrate
    start = datetime.datetime(2000, 1, 1)
    stop = start + datetime.timedelta(days=nday)
    time = numpy.arange(date2num(start), date2num(stop))
    return m.run(time, spinup=0, verbose=verbose, save_spinup=False, save_loss_rates=True, save_f=True, dt=1.)

def run_and_plot(ax, label=None, *args, **kwargs):
    result = run(verbose=False, *args, **kwargs)
    result.plot_spectrum(ax=ax, style='-', label=label)

In [ ]:
result = run(nday=1000, c_ini=1.)

In [ ]:
def start_stop(play=True):
    if play:
        anim.event_source.start()
    else:
        anim.event_source.stop()
anim = result.animate_spectrum(repeat=False)
widgets.interact(start_stop);

In [ ]:
fig, ax = pyplot.subplots()

In [ ]:
run_and_plot(ax, label='default')

In [ ]:
run_and_plot(ax, F=0, label='no fishing')

In [ ]:
run_and_plot(ax, z0pre=0., z_spre=0., label='no residual mortality')